In [1]:
import itertools
import networkx as nx
import numpy as np
from tqdm import tqdm
from collections import defaultdict
from itertools import chain
import sys
sys.path.append('/Users/ziniuwu/Desktop/research/BayesNet')
from Pgmpy.factors import factor_product
from Pgmpy.models import BayesianModel, JunctionTree
from Pgmpy.inference.EliminationOrder import (
    WeightedMinFill,
    MinNeighbors,
    MinFill,
    MinWeight,
)
from Pgmpy.factors.discrete import TabularCPD

In [2]:
import numpy as np
import pandas as pd
import time
import sys
import copy
from Models.pgmpy_BN import Pgmpy_BN
from Testing.toy_dataset import *

In [80]:
toy_df = toy_data_highly_correlated_cat(nrows=100000, return_df=True)

In [81]:
toy_df.head(5)

,id,cat_attr1,cat_attr2,cat_attr3,cat_attr4,cat_attr5,cat_attr6,cat_attr7,cat_attr8
0,0,5,5,10,10,20,15,15,50
1,1,0,0,0,0,0,0,10,10
2,2,3,3,6,6,12,8,13,34
3,3,3,4,7,9,16,15,13,41
4,4,7,7,14,14,28,17,17,66


In [82]:
BN = Pgmpy_BN('title')
BN.build_from_data(toy_df, algorithm="chow-liu", max_parents=2, ignore_cols=['id'], sample_size=50000)

In [83]:
class VariableElimination(object):
    def __init__(self, model):
        self.model = model
        model.check_model()

        if isinstance(model, JunctionTree):
            self.variables = set(chain(*model.nodes()))
        else:
            self.variables = model.nodes()

        self.cardinality = {}
        self.factors = defaultdict(list)

        if isinstance(model, BayesianModel):
            self.state_names_map = {}
            for node in model.nodes():
                cpd = model.get_cpds(node)
                if isinstance(cpd, TabularCPD):
                    self.cardinality[node] = cpd.variable_card
                    cpd = cpd.to_factor()
                for var in cpd.scope():
                    self.factors[var].append(cpd)
                self.state_names_map.update(cpd.no_to_name)

        elif isinstance(model, JunctionTree):
            self.cardinality = model.get_cardinality()

            for factor in model.get_factors():
                for var in factor.variables:
                    self.factors[var].append(factor)
    
    def _get_working_factors(self, evidence):
        """
        Uses the evidence given to the query methods to modify the factors before running
        the variable elimination algorithm.
        Parameters
        ----------
        evidence: dict
            Dict of the form {variable: state}
        Returns
        -------
        dict: Modified working factors.
        """

        working_factors = {
            node: {(factor, None) for factor in self.factors[node]}
            for node in self.factors
        }

        # Dealing with evidence. Reducing factors over it before VE is run.
        if evidence:
            for evidence_var in evidence:
                print(evidence_var)
                for factor, origin in working_factors[evidence_var]:
                    print(factor.scope())
                    factor_reduced = factor.reduce(
                        [(evidence_var, evidence[evidence_var])], inplace=False
                    )
                    print(factor_reduced.scope())
                    for var in factor_reduced.scope():
                        working_factors[var].remove((factor, origin))
                        working_factors[var].add((factor_reduced, evidence_var))
                if type(evidence[evidence_var]) != list:
                    del working_factors[evidence_var]
        return working_factors
    
    def _get_elimination_order(
        self, variables, evidence, elimination_order="minfill", show_progress=False
    ):
        """
        Deals with all elimination order parameters given to _variable_elimination method
        and returns a list of variables that are to be eliminated
        Parameters
        ----------
        elimination_order: str or list
        Returns
        -------
        list: A list of variables names in the order they need to be eliminated.
        """
        not_evidence_eliminate = []
        if evidence is not None:
            for key in evidence:
                if type(evidence[key]) != list:
                    not_evidence_eliminate.append(key)
        to_eliminate = (
            set(self.variables)
            - set(variables)
            - set(not_evidence_eliminate)
        )

        # Step 1: If elimination_order is a list, verify it's correct and return.
        if hasattr(elimination_order, "__iter__") and (
            not isinstance(elimination_order, str)
        ):
            if any(
                var in elimination_order
                for var in set(variables).union(
                    set(evidence.keys() if evidence else [])
                )
            ):
                raise ValueError(
                    "Elimination order contains variables which are in"
                    " variables or evidence args"
                )
            else:
                return elimination_order

        # Step 2: If elimination order is None or a Markov model, return a random order.
        elif (elimination_order is None) or (not isinstance(self.model, BayesianModel)):
            return to_eliminate

        # Step 3: If elimination order is a str, compute the order using the specified heuristic.
        elif isinstance(elimination_order, str) and isinstance(
            self.model, BayesianModel
        ):
            heuristic_dict = {
                "weightedminfill": WeightedMinFill,
                "minneighbors": MinNeighbors,
                "minweight": MinWeight,
                "minfill": MinFill,
            }
            elimination_order = heuristic_dict[elimination_order.lower()](
                self.model
            ).get_elimination_order(nodes=to_eliminate, show_progress=show_progress)
            return elimination_order
    
    def _variable_elimination(
        self,
        variables,
        operation,
        evidence=None,
        elimination_order="MinFill",
        joint=True,
        show_progress=False,
    ):
        """
        Implementation of a generalized variable elimination.

        Parameters
        ----------
        variables: list, array-like
            variables that are not to be eliminated.

        operation: str ('marginalize' | 'maximize')
            The operation to do for eliminating the variable.

        evidence: dict
            a dict key, value pair as {var: state_of_var_observed}
            None if no evidence

        elimination_order: str or list (array-like)
            If str: Heuristic to use to find the elimination order.
            If array-like: The elimination order to use.
            If None: A random elimination order is used.
        """
        # Step 1: Deal with the input arguments.
        if isinstance(variables, str):
            raise TypeError("variables must be a list of strings")
        if isinstance(evidence, str):
            raise TypeError("evidence must be a list of strings")

        # Dealing with the case when variables is not provided.
        if not variables:
            all_factors = []
            for factor_li in self.factors.values():
                all_factors.extend(factor_li)
            if joint:
                return factor_product(*set(all_factors))
            else:
                return set(all_factors)

        # Step 2: Prepare data structures to run the algorithm.
        eliminated_variables = set()
        # Get working factors and elimination order
        working_factors = self._get_working_factors(evidence)
        elimination_order = self._get_elimination_order(
            variables, evidence, elimination_order, show_progress=show_progress
        )

        # Step 3: Run variable elimination
        if show_progress:
            pbar = tqdm(elimination_order)
        else:
            pbar = elimination_order

        for var in pbar:
            if show_progress:
                pbar.set_description("Eliminating: {var}".format(var=var))
            # Removing all the factors containing the variables which are
            # eliminated (as all the factors should be considered only once)
            factors = [
                factor
                for factor, _ in working_factors[var]
                if not set(factor.variables).intersection(eliminated_variables)
            ]
            phi = factor_product(*factors)
            phi = getattr(phi, operation)([var], inplace=False)
            del working_factors[var]
            for variable in phi.variables:
                working_factors[variable].add((phi, var))
            eliminated_variables.add(var)

        # Step 4: Prepare variables to be returned.
        final_distribution = set()
        for node in working_factors:
            for factor, origin in working_factors[node]:
                if not set(factor.variables).intersection(eliminated_variables):
                    final_distribution.add((factor, origin))
        final_distribution = [factor for factor, _ in final_distribution]

        if joint:
            if isinstance(self.model, BayesianModel):
                return factor_product(*final_distribution).normalize(inplace=False)
            else:
                return factor_product(*final_distribution)
        else:
            query_var_factor = {}
            for query_var in variables:
                phi = factor_product(*final_distribution)
                query_var_factor[query_var] = phi.marginalize(
                    list(set(variables) - set([query_var])), inplace=False
                ).normalize(inplace=False)
            return query_var_factor

    def query(
        self,
        variables,
        evidence=None,
        elimination_order="MinFill",
        joint=True,
        show_progress=False,
    ):
        """
        Parameters
        ----------
        variables: list
            list of variables for which you want to compute the probability

        evidence: dict
            a dict key, value pair as {var: state_of_var_observed}
            None if no evidence

        elimination_order: list
            order of variable eliminations (if nothing is provided) order is
            computed automatically

        joint: boolean (default: True)
            If True, returns a Joint Distribution over `variables`.
            If False, returns a dict of distributions over each of the `variables`.
        """
        common_vars = set(evidence if evidence is not None else []).intersection(
            set(variables)
        )
        if common_vars:
            raise ValueError(
                f"Can't have the same variables in both `variables` and `evidence`. Found in both: {common_vars}"
            )

        return self._variable_elimination(
            variables=variables,
            operation="marginalize",
            evidence=evidence,
            elimination_order=elimination_order,
            joint=joint,
            show_progress=show_progress,
        )

In [84]:
ve = VariableElimination(BN.model)

In [127]:
tic = time.time()
q = ve.query(["cat_attr6"])
print(time.time()-tic)

0.08713388442993164


In [128]:
print(np.sum(q.values[[1,2,3]]))

0.15189


In [129]:
v = 0.4490*0.265718612153532*0.15189*100000
print(v)

1812.1639999999989


In [122]:
len(toy_df.query("cat_attr7 == 13 and cat_attr2 in [3, 9, 2] and cat_attr6 in [16, 12, 10]"))

1812

In [79]:
toy_df.head(20)

,id,cont_attr1,cont_attr2,cont_attr3,cont_attr4,cont_attr5,cont_attr6,cont_attr7,cont_attr8
0,0,35.281047,-9.675950,25.605097,93.202617,-19.189875,-6.742352,128.562684,128.235555
1,1,8.003144,25.761141,33.764285,25.007860,69.402853,70.043356,33.687760,206.898255
2,2,19.574760,-2.597574,16.977185,53.936899,-1.493936,2.337777,71.827292,89.648319
3,3,44.817864,-3.961568,40.856296,117.044660,-4.903919,15.182490,161.763259,212.898126
4,4,37.351160,-6.689752,30.661408,98.377900,-11.724380,2.226712,133.268569,154.432309
5,5,-19.545558,-7.828864,-27.374421,-43.863894,-14.572159,-10.751871,-65.387037,-118.085489
6,6,19.001768,-12.248124,6.753645,52.504421,-25.620309,-22.009637,72.710338,31.834038
7,7,-3.027144,-13.530479,-16.557623,-2.567860,-28.826198,-28.734562,-5.914102,-80.032486
8,8,-2.064377,26.544593,24.480216,-0.160943,71.361483,71.404099,-4.792016,162.453783
9,9,8.211970,-8.973909,-0.761939,25.529925,-17.434772,-16.760408,31.265455,-3.691664


In [32]:
tic = time.time()
order = ve._get_working_factors({"cont_attr2": [1,2,3], "cont_attr6": 2})
print(time.time()-tic)

cont_attr2
['cont_attr5', 'cont_attr2']
['cont_attr5', 'cont_attr2']
['cont_attr2', 'cont_attr6']
['cont_attr2', 'cont_attr6']
['cont_attr2', 'cont_attr6']
['cont_attr2', 'cont_attr6']
['cont_attr5', 'cont_attr2']
['cont_attr5', 'cont_attr2']
cont_attr6
['cont_attr6', 'cont_attr8']
['cont_attr8']
['cont_attr2', 'cont_attr6']
['cont_attr2']
0.07889318466186523


In [33]:
order

{'cont_attr8': {(<DiscreteFactor representing phi(cont_attr8:120) at 0x1a31e93a50>,
   'cont_attr6'),
  (<DiscreteFactor representing phi(cont_attr8:120, cont_attr3:120) at 0x1a33929850>,
   None)},
 'cont_attr2': {(<DiscreteFactor representing phi(cont_attr2:3) at 0x1a31e93d50>,
   'cont_attr6'),
  (<DiscreteFactor representing phi(cont_attr5:120, cont_attr2:3) at 0x1a31e93410>,
   'cont_attr2')},
 'cont_attr7': {(<DiscreteFactor representing phi(cont_attr3:120, cont_attr7:120) at 0x1a31f8ead0>,
   None),
  (<DiscreteFactor representing phi(cont_attr7:120, cont_attr1:120) at 0x1a31f8e790>,
   None)},
 'cont_attr1': {(<DiscreteFactor representing phi(cont_attr1:120) at 0x1a31f8e890>,
   None),
  (<DiscreteFactor representing phi(cont_attr4:120, cont_attr1:120) at 0x1a31f8eb90>,
   None),
  (<DiscreteFactor representing phi(cont_attr7:120, cont_attr1:120) at 0x1a31f8e790>,
   None)},
 'cont_attr3': {(<DiscreteFactor representing phi(cont_attr3:120, cont_attr7:120) at 0x1a31f8ead0>,
   N

In [15]:
BN.model.edges

OutEdgeView([('cont_attr6', 'cont_attr2'), ('cont_attr2', 'cont_attr5'), ('cont_attr7', 'cont_attr3'), ('cont_attr3', 'cont_attr8'), ('cont_attr1', 'cont_attr4'), ('cont_attr1', 'cont_attr7'), ('cont_attr8', 'cont_attr6')])

In [16]:
ve.factors

defaultdict(list,
            {'cont_attr6': [<DiscreteFactor representing phi(cont_attr6:120, cont_attr8:120) at 0x1097048d0>,
              <DiscreteFactor representing phi(cont_attr2:120, cont_attr6:120) at 0x10c602190>],
             'cont_attr8': [<DiscreteFactor representing phi(cont_attr6:120, cont_attr8:120) at 0x1097048d0>,
              <DiscreteFactor representing phi(cont_attr8:120, cont_attr3:120) at 0x1a2e3c2690>],
             'cont_attr2': [<DiscreteFactor representing phi(cont_attr2:120, cont_attr6:120) at 0x10c602190>,
              <DiscreteFactor representing phi(cont_attr5:120, cont_attr2:120) at 0x1a34531b50>],
             'cont_attr7': [<DiscreteFactor representing phi(cont_attr7:120, cont_attr1:120) at 0x1a34531150>,
              <DiscreteFactor representing phi(cont_attr3:120, cont_attr7:120) at 0x1a34531890>],
             'cont_attr1': [<DiscreteFactor representing phi(cont_attr7:120, cont_attr1:120) at 0x1a34531150>,
              <DiscreteFactor represent

In [ ]:
dsf = list(order["cont_attr6"])[1][0]

In [ ]:
dsf.values.shape

In [ ]:
slice_ = [slice(None)] * 2

In [ ]:
slice_[1] = [1,2,3,4]

In [ ]:
dsf.values[slice_].shape

In [ ]:
dsf.cardinality

In [ ]:
d = dsf.state_names

In [ ]:
del d["cont_attr2"]

In [ ]:
from __future__ import division

from itertools import product
from collections import namedtuple
from warnings import warn
import copy
import numpy as np

from Pgmpy.factors.base import BaseFactor
from Pgmpy.utils import StateNameMixin
from Pgmpy.extern import tabulate

State = namedtuple("State", ["var", "state"])


class DiscreteFactor(BaseFactor, StateNameMixin):
    """
    Base class for DiscreteFactor.
    """

    def __init__(self, variables, cardinality, values, state_names={}):
        """
        Initialize a factor class.

        Defined above, we have the following mapping from variable
        assignments to the index of the row vector in the value field:
        +-----+-----+-----+-------------------+
        |  x1 |  x2 |  x3 |    phi(x1, x2, x3)|
        +-----+-----+-----+-------------------+
        | x1_0| x2_0| x3_0|     phi.value(0)  |
        +-----+-----+-----+-------------------+
        | x1_0| x2_0| x3_1|     phi.value(1)  |
        +-----+-----+-----+-------------------+
        | x1_0| x2_1| x3_0|     phi.value(2)  |
        +-----+-----+-----+-------------------+
        | x1_0| x2_1| x3_1|     phi.value(3)  |
        +-----+-----+-----+-------------------+
        | x1_1| x2_0| x3_0|     phi.value(4)  |
        +-----+-----+-----+-------------------+
        | x1_1| x2_0| x3_1|     phi.value(5)  |
        +-----+-----+-----+-------------------+
        | x1_1| x2_1| x3_0|     phi.value(6)  |
        +-----+-----+-----+-------------------+
        | x1_1| x2_1| x3_1|     phi.value(7)  |
        +-----+-----+-----+-------------------+

        Parameters
        ----------
        variables: list, array-like
            List of variables in the scope of the factor.

        cardinality: list, array_like
            List of cardinalities of each variable. `cardinality` array must have a value
            corresponding to each variable in `variables`.

        values: list, array_like
            List of values of factor.
            A DiscreteFactor's values are stored in a row vector in the value
            using an ordering such that the left-most variables as defined in
            `variables` cycle through their values the fastest.

        """
        if isinstance(variables, str):
            raise TypeError("Variables: Expected type list or array like, got string")

        values = np.array(values, dtype=float)

        if len(cardinality) != len(variables):
            raise ValueError(
                "Number of elements in cardinality must be equal to number of variables"
            )

        if values.size != np.product(cardinality):
            raise ValueError(
                "Values array must be of size: {size}".format(
                    size=np.product(cardinality)
                )
            )

        if len(set(variables)) != len(variables):
            raise ValueError("Variable names cannot be same")

        self.variables = list(variables)
        self.cardinality = np.array(cardinality, dtype=int)
        self.values = values.reshape(self.cardinality)

        # Set the state names
        super(DiscreteFactor, self).store_state_names(
            variables, cardinality, state_names
        )

    def scope(self):
        """
        Returns the scope of the factor.

        Returns
        -------
        list: List of variable names in the scope of the factor.
        """
        return self.variables

    def get_cardinality(self, variables):
        """
        Returns cardinality of a given variable

        Parameters
        ----------
        variables: list, array-like
                A list of variable names.

        Returns
        -------
        dict: Dictionary of the form {variable: variable_cardinality}
        {'x1': 2, 'x2': 3}
        """
        if isinstance(variables, str):
            raise TypeError("variables: Expected type list or array-like, got type str")

        if not all([var in self.variables for var in variables]):
            raise ValueError("Variable not in scope")

        return {var: self.cardinality[self.variables.index(var)] for var in variables}

    def assignment(self, index):
        """
        Returns a list of assignments for the corresponding index.

        Parameters
        ----------
        index: list, array-like
            List of indices whose assignment is to be computed

        Returns
        -------
        list: Returns a list of full assignments of all the variables of the factor.
        """
        index = np.array(index)

        max_possible_index = np.prod(self.cardinality) - 1
        if not all(i <= max_possible_index for i in index):
            raise IndexError("Index greater than max possible index")

        assignments = np.zeros((len(index), len(self.scope())), dtype=np.int)
        rev_card = self.cardinality[::-1]
        for i, card in enumerate(rev_card):
            assignments[:, i] = index % card
            index = index // card

        assignments = assignments[:, ::-1]

        return [
            [
                (key, self.get_state_names(key, val))
                for key, val in zip(self.variables, values)
            ]
            for values in assignments
        ]

    def identity_factor(self):
        """
        Returns the identity factor.

        Def: The identity factor of a factor has the same scope and cardinality as the original factor,
             but the values for all the assignments is 1. When the identity factor is multiplied with
             the factor it returns the factor itself.

        Returns
        -------
        DiscreteFactor: The identity factor
        """
        return DiscreteFactor(
            variables=self.variables,
            cardinality=self.cardinality,
            values=np.ones(self.values.size),
            state_names=self.state_names,
        )

    def marginalize(self, variables, inplace=True):
        """
        Modifies the factor with marginalized values.

        Parameters
        ----------
        variables: list, array-like
            List of variables over which to marginalize.

        inplace: boolean
            If inplace=True it will modify the factor itself, else would return
            a new factor.

        Returns
        -------
        DiscreteFactor or None: if inplace=True (default) returns None
                        if inplace=False returns a new `DiscreteFactor` instance.
        """

        if isinstance(variables, str):
            raise TypeError("variables: Expected type list or array-like, got type str")

        phi = self if inplace else self.copy()

        for var in variables:
            if var not in phi.variables:
                raise ValueError("{var} not in scope.".format(var=var))

        var_indexes = [phi.variables.index(var) for var in variables]

        index_to_keep = sorted(set(range(len(self.variables))) - set(var_indexes))
        phi.variables = [phi.variables[index] for index in index_to_keep]
        phi.cardinality = phi.cardinality[index_to_keep]
        phi.del_state_names(variables)

        phi.values = np.sum(phi.values, axis=tuple(var_indexes))

        if not inplace:
            return phi

    def maximize(self, variables, inplace=True):
        """
        Maximizes the factor with respect to `variables`.

        Parameters
        ----------
        variables: list, array-like
            List of variables with respect to which factor is to be maximized

        inplace: boolean
            If inplace=True it will modify the factor itself, else would return
            a new factor.

        Returns
        -------
        DiscreteFactor or None: if inplace=True (default) returns None
                        if inplace=False returns a new `DiscreteFactor` instance.
        """
        if isinstance(variables, str):
            raise TypeError("variables: Expected type list or array-like, got type str")

        phi = self if inplace else self.copy()

        for var in variables:
            if var not in phi.variables:
                raise ValueError("{var} not in scope.".format(var=var))

        var_indexes = [phi.variables.index(var) for var in variables]

        index_to_keep = sorted(set(range(len(self.variables))) - set(var_indexes))
        phi.variables = [phi.variables[index] for index in index_to_keep]
        phi.cardinality = phi.cardinality[index_to_keep]
        phi.del_state_names(variables)

        phi.values = np.max(phi.values, axis=tuple(var_indexes))

        if not inplace:
            return phi

    def normalize(self, inplace=True):
        """
        Normalizes the values of factor so that they sum to 1.

        Parameters
        ----------
        inplace: boolean
            If inplace=True it will modify the factor itself, else would return
            a new factor

        Returns
        -------
        DiscreteFactor or None: if inplace=True (default) returns None
                        if inplace=False returns a new `DiscreteFactor` instance.
        """
        phi = self if inplace else self.copy()

        phi.values = phi.values / phi.values.sum()

        if not inplace:
            return phi

    def reduce(self, values, inplace=True):
        """
        Reduces the factor to the context of given variable values.

        Parameters
        ----------
        values: list, array-like
            A list of tuples of the form (variable_name, variable_state).

        inplace: boolean
            If inplace=True it will modify the factor itself, else would return
            a new factor.

        Returns
        -------
        DiscreteFactor or None: if inplace=True (default) returns None
                        if inplace=False returns a new `DiscreteFactor` instance.
        """
        # Check if values is an array
        if isinstance(values, str):
            raise TypeError("values: Expected type list or array-like, got type str")

        if not all([isinstance(state_tuple, tuple) for state_tuple in values]):
            raise TypeError(
                "values: Expected type list of tuples, get type {type}", type(values[0])
            )

        # Check if all variables in values are in the factor
        for var, _ in values:
            if var not in self.variables:
                raise ValueError(f"The variable: {var} is not in the factor")

        phi = self if inplace else self.copy()

        # Convert the state names to state number. If state name not found treat them as
        # state numbers.
        try:
            for i, (var, state_name) in enumerate(values):
                if type(state_name) == list:
                    values[i] = (var, [self.get_state_no(var, no) for no in state_name])
                else:
                    values[i] = (var, self.get_state_no(var, state_name))
        except KeyError:
            warn(
                "Found unknown state name. Trying to switch to using all state names as state numbers"
            )
            
        var_index_to_del = []
        slice_ = [slice(None)] * len(self.variables)
        point_query = True
        cardinality = phi.cardinality
        state_names = phi.state_names
        del_state_names = []
        for var, state in values:
            var_index = phi.variables.index(var)
            slice_[var_index] = state
            if type(state) == list:
                point_query = False
                cardinality[var_index] = len(state)
                state_names[var] = state
            else:
                var_index_to_del.append(var_index)
                del_state_names.append(var)

        var_index_to_keep = sorted(
            set(range(len(phi.variables))) - set(var_index_to_del)
        )
        # set difference is not gaurenteed to maintain ordering
        phi.variables = [phi.variables[index] for index in var_index_to_keep]
        phi.cardinality = cardinality[var_index_to_keep]
        phi.del_state_names(del_state_names)
        phi.values = phi.values[tuple(slice_)]

        if not point_query:
            super(DiscreteFactor, phi).store_state_names(
                phi.variables, cardinality, state_names
            )

        if not inplace:
            return phi

    def sum(self, phi1, inplace=True):
        """
        DiscreteFactor sum with `phi1`.

        Parameters
        ----------
        phi1: `DiscreteFactor` instance.
            DiscreteFactor to be added.

        inplace: boolean
            If inplace=True it will modify the factor itself, else would return
            a new factor.

        Returns
        -------
        DiscreteFactor or None: if inplace=True (default) returns None
                        if inplace=False returns a new `DiscreteFactor` instance.
        """
        phi = self if inplace else self.copy()
        if isinstance(phi1, (int, float)):
            phi.values += phi1
        else:
            phi1 = phi1.copy()

            # modifying phi to add new variables
            extra_vars = set(phi1.variables) - set(phi.variables)
            if extra_vars:
                slice_ = [slice(None)] * len(phi.variables)
                slice_.extend([np.newaxis] * len(extra_vars))
                phi.values = phi.values[tuple(slice_)]

                phi.variables.extend(extra_vars)

                new_var_card = phi1.get_cardinality(extra_vars)
                phi.cardinality = np.append(
                    phi.cardinality, [new_var_card[var] for var in extra_vars]
                )

            # modifying phi1 to add new variables
            extra_vars = set(phi.variables) - set(phi1.variables)
            if extra_vars:
                slice_ = [slice(None)] * len(phi1.variables)
                slice_.extend([np.newaxis] * len(extra_vars))
                phi1.values = phi1.values[tuple(slice_)]

                phi1.variables.extend(extra_vars)
                # No need to modify cardinality as we don't need it.

            # rearranging the axes of phi1 to match phi
            for axis in range(phi.values.ndim):
                exchange_index = phi1.variables.index(phi.variables[axis])
                phi1.variables[axis], phi1.variables[exchange_index] = (
                    phi1.variables[exchange_index],
                    phi1.variables[axis],
                )
                phi1.values = phi1.values.swapaxes(axis, exchange_index)

            phi.values = phi.values + phi1.values

        if not inplace:
            return phi

    def product(self, phi1, inplace=True):
        """
        DiscreteFactor product with `phi1`.

        Parameters
        ----------
        phi1: `DiscreteFactor` instance
            DiscreteFactor to be multiplied.

        inplace: boolean
            If inplace=True it will modify the factor itself, else would return
            a new factor.

        Returns
        -------
        DiscreteFactor or None: if inplace=True (default) returns None
                        if inplace=False returns a new `DiscreteFactor` instance.
        """
        phi = self if inplace else self.copy()
        if isinstance(phi1, (int, float)):
            phi.values *= phi1
        else:
            phi1 = phi1.copy()

            # modifying phi to add new variables
            extra_vars = set(phi1.variables) - set(phi.variables)
            if extra_vars:
                slice_ = [slice(None)] * len(phi.variables)
                slice_.extend([np.newaxis] * len(extra_vars))
                phi.values = phi.values[tuple(slice_)]

                phi.variables.extend(extra_vars)

                new_var_card = phi1.get_cardinality(extra_vars)
                phi.cardinality = np.append(
                    phi.cardinality, [new_var_card[var] for var in extra_vars]
                )

            # modifying phi1 to add new variables
            extra_vars = set(phi.variables) - set(phi1.variables)
            if extra_vars:
                slice_ = [slice(None)] * len(phi1.variables)
                slice_.extend([np.newaxis] * len(extra_vars))
                phi1.values = phi1.values[tuple(slice_)]

                phi1.variables.extend(extra_vars)
                # No need to modify cardinality as we don't need it.

            # rearranging the axes of phi1 to match phi
            for axis in range(phi.values.ndim):
                exchange_index = phi1.variables.index(phi.variables[axis])
                phi1.variables[axis], phi1.variables[exchange_index] = (
                    phi1.variables[exchange_index],
                    phi1.variables[axis],
                )
                phi1.values = phi1.values.swapaxes(axis, exchange_index)

            phi.values = phi.values * phi1.values
            phi.add_state_names(phi1)

        if not inplace:
            return phi

    def divide(self, phi1, inplace=True):
        """
        DiscreteFactor division by `phi1`.

        Parameters
        ----------
        phi1 : `DiscreteFactor` instance
            The denominator for division.

        inplace: boolean
            If inplace=True it will modify the factor itself, else would return
            a new factor.

        Returns
        -------
        DiscreteFactor or None: if inplace=True (default) returns None
                        if inplace=False returns a new `DiscreteFactor` instance.
        """
        phi = self if inplace else self.copy()
        phi1 = phi1.copy()

        if set(phi1.variables) - set(phi.variables):
            raise ValueError("Scope of divisor should be a subset of dividend")

        # Adding extra variables in phi1.
        extra_vars = set(phi.variables) - set(phi1.variables)
        if extra_vars:
            slice_ = [slice(None)] * len(phi1.variables)
            slice_.extend([np.newaxis] * len(extra_vars))
            phi1.values = phi1.values[tuple(slice_)]

            phi1.variables.extend(extra_vars)

        # Rearranging the axes of phi1 to match phi
        for axis in range(phi.values.ndim):
            exchange_index = phi1.variables.index(phi.variables[axis])
            phi1.variables[axis], phi1.variables[exchange_index] = (
                phi1.variables[exchange_index],
                phi1.variables[axis],
            )
            phi1.values = phi1.values.swapaxes(axis, exchange_index)

        phi.values = phi.values / phi1.values

        # If factor division 0/0 = 0 but is undefined for x/0. In pgmpy we are using
        # np.inf to represent x/0 cases.
        phi.values[np.isnan(phi.values)] = 0

        if not inplace:
            return phi

    def copy(self):
        """
        Returns a copy of the factor.

        Returns
        -------
        DiscreteFactor: copy of the factor
        """
        # not creating a new copy of self.values and self.cardinality
        # because __init__ methods does that.
        return DiscreteFactor(
            self.scope(),
            self.cardinality,
            self.values,
            state_names=self.state_names.copy(),
        )

    def is_valid_cpd(self):
        return np.allclose(
            self.to_factor()
            .marginalize(self.scope()[:1], inplace=False)
            .values.flatten("C"),
            np.ones(np.product(self.cardinality[:0:-1])),
            atol=0.01,
        )

    def __str__(self):
        return self._str(phi_or_p="phi", tablefmt="grid")

    def _str(self, phi_or_p="phi", tablefmt="grid", print_state_names=True):
        """
        Generate the string from `__str__` method.

        Parameters
        ----------
        phi_or_p: 'phi' | 'p'
                'phi': When used for Factors.
                  'p': When used for CPDs.
        print_state_names: boolean
                If True, the user defined state names are displayed.
        """
        string_header = list(map(str, self.scope()))
        string_header.append(
            "{phi_or_p}({variables})".format(
                phi_or_p=phi_or_p, variables=",".join(string_header)
            )
        )

        value_index = 0
        factor_table = []
        for prob in product(*[range(card) for card in self.cardinality]):
            if self.state_names and print_state_names:
                prob_list = [
                    "{var}({state})".format(
                        var=list(self.variables)[i],
                        state=self.state_names[list(self.variables)[i]][prob[i]],
                    )
                    for i in range(len(self.variables))
                ]
            else:
                prob_list = [
                    "{s}_{d}".format(s=list(self.variables)[i], d=prob[i])
                    for i in range(len(self.variables))
                ]

            prob_list.append(self.values.ravel()[value_index])
            factor_table.append(prob_list)
            value_index += 1

        return tabulate(
            factor_table, headers=string_header, tablefmt=tablefmt, floatfmt=".4f"
        )

    def __repr__(self):
        var_card = ", ".join(
            [
                "{var}:{card}".format(var=var, card=card)
                for var, card in zip(self.variables, self.cardinality)
            ]
        )
        return "<DiscreteFactor representing phi({var_card}) at {address}>".format(
            address=hex(id(self)), var_card=var_card
        )

    def __mul__(self, other):
        return self.product(other, inplace=False)

    def __rmul__(self, other):
        return self.__mul__(other)

    def __add__(self, other):
        return self.sum(other, inplace=False)

    def __radd__(self, other):
        return self.__add__(other)

    def __truediv__(self, other):
        return self.divide(other, inplace=False)

    __div__ = __truediv__

    def __eq__(self, other):
        if not (isinstance(self, DiscreteFactor) and isinstance(other, DiscreteFactor)):
            return False

        elif set(self.scope()) != set(other.scope()):
            return False

        else:
            phi = other.copy()
            for axis in range(self.values.ndim):
                exchange_index = phi.variables.index(self.variables[axis])
                phi.variables[axis], phi.variables[exchange_index] = (
                    phi.variables[exchange_index],
                    phi.variables[axis],
                )
                phi.cardinality[axis], phi.cardinality[exchange_index] = (
                    phi.cardinality[exchange_index],
                    phi.cardinality[axis],
                )
                phi.values = phi.values.swapaxes(axis, exchange_index)

            if phi.values.shape != self.values.shape:
                return False
            elif not np.allclose(phi.values, self.values):
                return False
            elif not all(self.cardinality == phi.cardinality):
                return False
            elif not (self.state_names == phi.state_names):
                return False
            else:
                return True

    def __ne__(self, other):
        return not self.__eq__(other)

    def __hash__(self):
        variable_hashes = [hash(variable) for variable in self.variables]
        sorted_var_hashes = sorted(variable_hashes)
        state_names_hash = hash(frozenset(self.state_names))
        phi = self.copy()
        for axis in range(phi.values.ndim):
            exchange_index = variable_hashes.index(sorted_var_hashes[axis])
            variable_hashes[axis], variable_hashes[exchange_index] = (
                variable_hashes[exchange_index],
                variable_hashes[axis],
            )
            phi.cardinality[axis], phi.cardinality[exchange_index] = (
                phi.cardinality[exchange_index],
                phi.cardinality[axis],
            )
            phi.values = phi.values.swapaxes(axis, exchange_index)
        return hash(
            str(sorted_var_hashes)
            + str(phi.values)
            + str(phi.cardinality)
            + str(state_names_hash)
        )


In [ ]:
phi = DiscreteFactor(['x1', 'x2', 'x3'], [2, 3, 4], np.ones(24))

In [ ]:
import time
tic = time.time()
n = phi.reduce([('x1', 0), ('x2',[0,2])], inplace=False)
print(time.time()-tic)

In [ ]:
print(n)